Following [this](https://pytorch-lightning.readthedocs.io/en/stable/model/train_model_basic.html)

In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import pytorch_lightning as pl  # NOTE.


# Define actual nn.Module's.

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        return self.l1(x)

# Define a LightningModule.

class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):  # NOTE.
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):  # NOTE.
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# Define a DataLoader.

dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset)

# ... Aaand train.

# model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train model
trainer = pl.Trainer(
    # For the sake of example's speed:
    max_epochs=2,
    limit_train_batches=100
)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)  # NOTE: .fit()


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /mnt/data-linux/Dropbox/Programming/wsl_repos/practice_py/lightning_tutorials/2_level_up/1_basic/MNIST/raw/train-images-idx3-ubyte.gz to /mnt/data-linux/Dropbox/Programming/wsl_repos/practice_py/lightning_tutorials/2_level_up/1_basic/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /mnt/data-linux/Dropbox/Programming/wsl_repos/practice_py/lightning_tutorials/2_level_up/1_basic/MNIST/raw/train-labels-idx1-ubyte.gz to /mnt/data-linux/Dropbox/Programming/wsl_repos/practice_py/lightning_tutorials/2_level_up/1_basic/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /mnt/data-linux/Dropbox/Programming/wsl_repos/practice_py/lightning_tutorials/2_level_up/1_basic/MNIST/raw/t10k-labels-idx1-ubyte.gz to /mnt/data-linux/Dropbox/Programming/wsl_repos/practice_py/lightning_tutorials/2_level_up/1_basic/MNIST/raw



GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/data-linux/miniconda3/envs/learn_py38_lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Missing logger folder: /mnt/data-linux/Dropbox/Programming/wsl_repos/practice_py/lightning_tutorials/2_level_up/1_basic/lightning_logs

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 50.4 K
1 | decoder | Decoder | 51.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
/mnt/data-linux/miniconda3/envs/learn_py38_lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: Possib

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


Under the hood, the Lightning Trainer runs the following training loop on your behalf:

```python
autoencoder = LitAutoEncoder(Encoder(), Decoder())
optimizer = autoencoder.configure_optimizers()

for batch_idx, batch in enumerate(train_loader):
    loss = autoencoder.training_step(batch, batch_idx)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
```

The power of Lightning comes when the training loop gets complicated as you add:
* validation/test splits, 
* schedulers, 
* distributed training 
* and all the latest SOTA techniques.

With Lightning, you can add mix all these techniques together without needing to rewrite a new loop every time.